# Micro partitions

Snowflake stores a table in many small chunks called micro-partitions. 
For each micro-partition, Snowflake keeps metadata like the MIN and MAX values of each column inside that chunk.
 
 * 50-500 MB of uncompressed data
 * Data stored in columnar format, not rows.
 * Repetition of column range of data can happen, causing ***overlapping***
  

## Overlapping micro-partitions

If we take on one column (example: order_date), each micro-partition has a range:

Micro-partition P1 contain dates from Jan 1 to Jan 10
Micro-partition P2 contain dates from Jan 11 to Jan 20

That is 0 data overlapping.

However, 
P1: [Jan 01 … Jan 20]
P2: [Jan 10 … Feb 05]
P3: [Jan 15 … Jan 25]

Jan 15 apperrs in the range of P1, P2 and P3 causing range overlap.
So, in case you search for order_date = jan 15. all 3 partitions will be scanned.


## Logical vs Physical Structure

![image info](https://docs.snowflake.com/en/_images/tables-clustered1.png)


In [ ]:
%%sql -r dataframe_2

use role sysadmin;
create or replace database sf_cert_prep;

--creating a table, forcing to be order by o_comment, even knowing all searches will be on o_orderdate.
create or replace table sf_cert_prep.public.t_orders_bad
as 
select * from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.orders order by o_comment  asc ;


--creating a table, forcing to be order by o_orderdate
create or replace table sf_cert_prep.public.t_orders_good
as 
select * from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.orders order by o_orderdate  asc ;

--creating a table, forcing to be order by o_comment, even knowing all searches will be on o_orderdate, but adding cluster key as orderdate
create or replace table sf_cert_prep.public.t_orders_bad_but_clustered 
cluster by (o_orderdate)
as 
select * from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.orders order by o_comment  asc ;


In [ ]:
%%sql -r dataframe_6

ALTER SESSION SET USE_CACHED_RESULT = FALSE; -- disabling query cache
-- cleaning warehouse cache
ALTER WAREHOUSE COMPUTE_WH SUSPEND;
ALTER WAREHOUSE COMPUTE_WH RESUME;

select * from sf_cert_prep.public.t_orders_bad where o_orderdate = '1993-09-08';
select --OPERATOR_STATISTICS,
OPERATOR_STATISTICS:pruning.partitions_scanned::integer as partitions_scanned,
OPERATOR_STATISTICS:pruning.partitions_total::integer  as partitions_total , 
partitions_scanned/partitions_total as overlap from table(get_query_operator_stats(last_query_id(-1))) where operator_type = 'TableScan';

In [ ]:
%%sql -r dataframe_11


ALTER SESSION SET USE_CACHED_RESULT = FALSE; -- disabling query cache
-- cleaning warehouse cache
ALTER WAREHOUSE COMPUTE_WH SUSPEND;
ALTER WAREHOUSE COMPUTE_WH RESUME;

select * from sf_cert_prep.public.t_orders_good where o_orderdate = '1993-09-08';
select --OPERATOR_STATISTICS,
OPERATOR_STATISTICS:pruning.partitions_scanned::integer as partitions_scanned,
OPERATOR_STATISTICS:pruning.partitions_total::integer  as partitions_total , 
partitions_scanned/partitions_total as overlap from table(get_query_operator_stats(last_query_id(-1))) where operator_type = 'TableScan';

In [ ]:
%%sql -r dataframe_14


ALTER SESSION SET USE_CACHED_RESULT = FALSE; -- disabling query cache
-- cleaning warehouse cache
ALTER WAREHOUSE COMPUTE_WH SUSPEND;
ALTER WAREHOUSE COMPUTE_WH RESUME;

select * from sf_cert_prep.public.t_orders_bad_but_clustered where o_orderdate = '1993-09-08';
select --OPERATOR_STATISTICS,
OPERATOR_STATISTICS:pruning.partitions_scanned::integer as partitions_scanned,
OPERATOR_STATISTICS:pruning.partitions_total::integer  as partitions_total , 
partitions_scanned/partitions_total as overlap from table(get_query_operator_stats(last_query_id(-1))) where operator_type = 'TableScan';

### Explanation

Usually we dont have the luxury to rewrite a production table the way we inteded. Merge,updates or reloads of the source, will mix up and mess up the way partition were "optimized".
As alternative for those cases, we can use "clustering keys". Clustering keys will create new partitions based on the keys or expressions that we provide.
* Old paritions will remain unchanged, if not used anymore, they will be marked to be deleted after timetravel / fail safe period
* New partitons will have priority in the metadata / query plan
* Re-clustering will not recreate all paritions, only the ones that will benefit the most


## clustering_information function break down.

Returns clustering information, including average clustering depth, for a table based on one or more columns in the table.

### Arguments:

1 - table_name - Table for which you want to return clustering information. 

2 - expression or columns:

Column names or expressions for which clustering information is returned:

- no clustering key -> this argument is required. If this argument is omitted, an error is returned.

- with clustering key -> optional; 


***Even if only one column name or expression is passed, it must be inside parentheses.***

3 - number_of_errors

Number of clustering errors returned by the function. If this argument is omitted, the 10 most recent errors are returned


#### List of Returns - not all listed, only most important ones. -> check documentation for full list: https://docs.snowflake.com/en/sql-reference/functions/system_clustering_information

* cluster_by_keys -> Columns in table used to return clustering information
* total_partition_count -> Total number of micro-partitions that comprise the table.
* total_constant_partition_count -> Total number of micro-partitions for which the value of the specified columns have reached a constant state. The most constant MP is, more efficient is the prunning.
* average_overlaps -> Average number of overlapping micro-partitions for each micro-partition in the table. A high number indicates the table is not well-clustered.
* average_depth -> Average overlap depth of each micro-partition in the table. A high number indicates the table is not well-clustered.
* partition_depth_histogram -> A histogram depicting the distribution of overlap depth for each micro-partition in the table



***Important***
IF the table has more than 2 million partitions this analysis is based in a subset.






In [ ]:
%%sql -r dataframe_12
select system$clustering_information('sf_cert_prep.public.t_orders_bad', '(o_orderdate)') as clust_info

In [ ]:
%%sql -r dataframe_16
select TABLE_NAME, CLUSTERING_KEY, AUTO_CLUSTERING_ON from sf_cert_prep.information_schema.tables where table_schema = 'PUBLIC';

In [ ]:
%%sql -r dataframe_8
with clust_info as 
(
select 'T_ORDERS_BAD' AS TABLE_NAME, parse_json(system$clustering_information('sf_cert_prep.public.T_ORDERS_BAD', '(o_orderdate)'))::variant as c_info
UNION ALL
select 'T_ORDERS_GOOD' AS TABLE_NAME,parse_json(system$clustering_information('sf_cert_prep.public.T_ORDERS_GOOD', '(o_orderdate)'))::variant as c_info
UNION ALL
select 'T_ORDERS_BAD_BUT_CLUSTERED' AS TABLE_NAME,parse_json(system$clustering_information('sf_cert_prep.public.T_ORDERS_BAD_BUT_CLUSTERED', '(o_orderdate)'))::variant as c_info
)

select 
TABLE_NAME,
c_info:cluster_by_keys as cluster_by_keys,
c_info:total_partition_count as total_partition_count,
c_info:total_constant_partition_count as total_constant_partition_count,
c_info:average_depth as average_depth,
c_info:average_overlaps as average_overlaps,
c_info:notes as notes,
c_info  from clust_info
;

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:

table_name = 'sf_cert_prep.public.t_orders_bad'
#table_name = 'sf_cert_prep.public.t_orders_good'
#table_name = 'sf_cert_prep.public.t_orders_bad_but_clustered'

column_name = '(o_orderdate)'

query = f'''
with clust_info as 
(select parse_json(system$clustering_information(\'{table_name}\', \'{column_name}\'))::variant as c_info),

hist as
(select c_info:partition_depth_histogram as partition_depth_histogram from clust_info )

SELECT  
  TRY_TO_NUMBER(f.key::string)                 AS partition_depth,       -- transform to in int for the histogram chart
  f.value::number                              AS count
FROM hist,
LATERAL FLATTEN(input => partition_depth_histogram) AS f
ORDER BY partition_depth NULLS LAST;
'''

# print(query)

df_hist = session.sql(query).to_pandas()  # Snowpark -> Pandas


fig = px.bar(
    df_hist,
    x="PARTITION_DEPTH",
    y="COUNT",
    text="COUNT",
    title="Partition Depth Histogram (Aggregated)",
    labels={"PARTITION_DEPTH": "Partition Depth", "COUNT": "Count"},
)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(xaxis=dict(dtick=1), yaxis_title="Count", bargap=0.15)
fig.show()



### Explanation

***Important!!***

There are 24 partitions in total.

From the 16th bucket, Snowflake starts incrmenting the depth number as twice the width of the previous bucket.
So in the t_orders_bad case, it will shown that all the 24 micro-partitions as depth 32, but in reality it should be 23 depth.


### Number of Overlap vs Overlap depth

![image info](https://docs.snowflake.com/en/_images/tables-clustering-ratio.png)


In [ ]:

table_name = 'sf_cert_prep.public.t_orders_bad_but_clustered'
column_name = '(o_orderdate)'

query = f'''
with clust_info as 
(select parse_json(system$clustering_information(\'{table_name}\', \'{column_name}\'))::variant as c_info),

hist as
(select c_info:partition_depth_histogram as partition_depth_histogram from clust_info )

SELECT  
  TRY_TO_NUMBER(f.key::string)                 AS partition_depth,       -- transform to in int for the histogram chart
  f.value::number                              AS count
FROM hist,
LATERAL FLATTEN(input => partition_depth_histogram) AS f
ORDER BY partition_depth NULLS LAST;
'''

# print(query)

df_hist = session.sql(query).to_pandas()  # Snowpark -> Pandas


fig = px.bar(
    df_hist,
    x="PARTITION_DEPTH",
    y="COUNT",
    text="COUNT",
    title="Partition Depth Histogram (Aggregated)",
    labels={"PARTITION_DEPTH": "Partition Depth", "COUNT": "Count"},
)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(xaxis=dict(dtick=1), yaxis_title="Count", bargap=0.15)
fig.show()



## Automatic Clustering

### Basics:
* it will not start start automatically, Snowflake only reclusters a clustered table if it will benefit from the operation.
* Snowflake performs automatic reclustering in the background, and you do not need to specify a warehouse to use. (serverless compute resources))
* Non-blocking DML

### Costs:

Compute -> The more the table is changed, higher the costs.
Storage -> reoganization will increase storage costs, but once the timetravel/failsafe period expires, old partitions will be dropped, causing "almost a net 0".

In [ ]:
%%sql -r dataframe_1


use sf_cert_prep.public;
ALTER TABLE t_orders_bad_but_clustered SUSPEND RECLUSTER;
SHOW TABLES LIKE 't_orders_bad_but_clustered';


In [ ]:
%%sql -r dataframe_19
use sf_cert_prep.public;
ALTER TABLE t_orders_bad_but_clustered RESUME  RECLUSTER;
SHOW TABLES LIKE 't_orders_bad_but_clustered';

In [ ]:
%%sql -r dataframe_20
-- data comes from snowflake.account_usage, so expect latency of 45m to 3h


SELECT TO_DATE(start_time) AS date,
  database_name,
  schema_name,
  table_name,
  SUM(credits_used) AS credits_used
FROM snowflake.account_usage.automatic_clustering_history
WHERE start_time >= DATEADD(month,-1,CURRENT_TIMESTAMP())
GROUP BY 1,2,3,4
ORDER BY 5 DESC;

### ESTIMATE_AUTOMATIC_CLUSTERING_COSTS

- it will return 2 costs, the intial value and the maintenance.
- If table created recently the maintenance estimation will return a comment saying that is best have at least 7 days to have a proper estimation
- uses samples from a subset of micro-paritions, so it is not guaranteed that will be the accurate
- for extra accurancy, needs to run multiple times and use the average of the results


In [ ]:

from snowflake.snowpark.context import get_active_session
session = get_active_session()

table_name = 'sf_cert_prep.public.t_orders_bad'
column_name = '(o_orderdate)'

query = f'''
SELECT SYSTEM$ESTIMATE_AUTOMATIC_CLUSTERING_COSTS(\'{table_name}\', \'{column_name}\'); '''

#print(query)
session.sql(query)



In [ ]:
%%sql -r dataframe_22
SELECT SYSTEM$ESTIMATE_AUTOMATIC_CLUSTERING_COSTS('sf_cert_prep.public.t_orders_bad_but_clustered', '(o_orderdate)');